In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
%cd /content/drive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

/content/drive/MyDrive/models/research


In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/drive/MyDrive/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/drive/MyDrive/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Install the Object Detection API
!pip install /content/drive/MyDrive/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 80.1 MB/s eta 0:00:00
     ━━

In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/drive/MyDrive/models/research/object_detection/builders/model_builder_tf2_test.py


/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

#Initilisation

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="j3BRFx7mf0UppDmjCBg5")
project = rf.workspace("xavierspace").project("yolotest-stofa")
dataset = project.version(3).download("tfrecord")


In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3692, done.
remote: Counting objects: 100% (3692/3692), done.
remote: Compressing objects: 100% (3079/3079), done.
remote: Total 3692 (delta 990), reused 1545 (delta 562), pack-reused 0
Receiving objects: 100% (3692/3692), 48.76 MiB | 17.64 MiB/s, done.
Resolving deltas: 100% (990/990), done.
Updating files: 100% (3354/3354), done.


In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/drive/MyDrive/models/research/object_detection/builders/model_builder_tf2_test.py

# Training code

In [ ]:
train_record_fname = '/content/drive/MyDrive/yolotest-3/train/vehicles.tfrecord'
val_record_fname = '/content/drive/MyDrive/yolotest-3/valid/vehicles.tfrecord'
label_map_pbtxt_fname = '/content/drive/MyDrive/yolotest-3/train/vehicles_label_map.pbtxt'

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
%ls
%cd /content/drive/MyDrive/models/mymodel/


 calculate_map_cartucho.py   labelmap.txt   test/
'Colab Notebooks'/           mAP/           training/
 custom_model_lite/          models/        yolotest-3/
/content/drive/MyDrive/models/mymodel


In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/drive/MyDrive/models/mymodel/
%cd /content/drive/MyDrive/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/drive/MyDrive/models/mymodel
--2023-03-11 11:38:31--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.202.128, 2607:f8b0:400e:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.202.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M  86.4MB/s    in 0.5s    

2023-03-11 11:38:32 (86.4 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]

--2023-03-11 11:38:33--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, 

In [ ]:
# Set training parameters for the model
num_steps = 40000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/content/drive/MyDrive/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/drive/MyDrive/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


Total classes: 3


In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/drive/MyDrive/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)
      
      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)
    
    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


/content/drive/MyDrive/models/mymodel
writing custom configuration file


In [ ]:
pipeline_file = '/content/drive/MyDrive/models/mymodel/pipeline_file.config'
model_dir = '/content/drive/MyDrive/training/'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/training/train'

In [ ]:
# Run training!
!python /content/drive/MyDrive/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

In [ ]:
# Make a directory to store the trained TFLite model
!mkdir /content/drive/MyDrive/custom_model_lite
output_directory = '/content/drive/MyDrive/custom_model_lite'

# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = '/content/drive/MyDrive/training'

!python /content/drive/MyDrive/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}


/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

# Test/Detect

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/custom_model_lite/saved_model')
tflite_model = converter.convert()

with open('/content/drive/MyDrive/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/drive/MyDrive/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape 
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])]
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) 
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) 
              label_ymin = max(ymin, labelSize[1] + 10) 
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) 

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

      
      if txt_only == False: 
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()
      
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)      
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

In [ ]:
# Set up variables for running user's model
PATH_TO_IMAGES='/content/drive/MyDrive/test'   # Path to test images folder
PATH_TO_MODEL='/content/drive/MyDrive/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/drive/MyDrive/labelmap.txt'   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 10   # Number of images to run detection on

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)



# Pipeline testinggggggggggg

ORB PIPELINE

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/custom_model_lite/saved_model')
tflite_model = converter.convert()

with open('/content/drive/MyDrive/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_video(modelpath, vidpath, lblpath, min_conf=0.5, savepath='/content/drive/MyDrive/results'):


  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  video=cv2.VideoCapture(vidpath)
  imW = video.get(cv2.CAP_PROP_FRAME_WIDTH)
  imH = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  fps = video.get(cv2.CAP_PROP_FPS) 
  out = cv2.VideoWriter('output_video3.mp4', fourcc, fps, (int(imW), int(imH)), isColor=True)

  #ORB algorithm parameters
  orb=cv2.ORB_create(fastThreshold=0, edgeThreshold=0)
  #bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  index_params= dict(algorithm = 6,
                   table_number = 6,
                   key_size = 12,
                   multi_probe_level = 1)
  search_params=dict(checks=32)
  matcher = cv2.FlannBasedMatcher(index_params, search_params)
  prev=[]
  vehicle_count = [0 for _ in range (3) ]
  count=0
  # Loop over every image and perform detection
  while(video.isOpened()):
      detected_count = [0 for _ in range (3) ]
      curr=[]
      ret,frame = video.read()
      if not ret:
        print("End of video reached!")
        break
      framec = frame.copy()      
      frame_rgb = cv2.cvtColor(framec, cv2.COLOR_BGR2RGB)
      frame_resized = cv2.resize(frame_rgb, (width, height))
      input_data = np.expand_dims(frame_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              cv2.rectangle(framec, (xmin,ymin), (xmax,ymax), (10, 255, 0), 4)
              #crop the object from the image
              cropped = frame[ymin:ymax,xmin:xmax]
              #cv2.imwrite("frame%d.jpg" % count, cropped) 
              keypoints, descriptors = orb.detectAndCompute(cropped,None)                      
              curr.append({
                  "descriptors" : descriptors,
                  "class": int(classes[i]),
                 "matched": False
              })
              #cropped = cv2.cvtColor(cropped,cv2.COLOR_BGR2RGB)
              #plt.figure(figsize=(12,16))
              #plt.imshow(cropped)
              #plt.show()
              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(framec, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(framec, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([classes[i], scores[i], xmin, ymin, xmax, ymax])
      framec = cv2.cvtColor(framec,cv2.COLOR_BGR2RGB)
      if prev:
        for ii in prev:
            for index , jj in enumerate(curr):
                try:
                  matches = matcher.knnMatch(jj['descriptors'],ii['descriptors'], 2)
                  good_matches=[]
                  for m in matches:
                      if len(m)==2 and m[0].distance < m[1].distance * 0.75:
                        good_matches.append(m)
                    #print(good_matches)
                  if len(good_matches)>=10:
                      curr[index]["matched"]=True
                except:
                  continue
      for ee in curr:
        if not ee["matched"]:
            vehicle_count[ee["class"]] += 1
      prev=curr       
      out.write(framec)
      for i , det in enumerate(detections):
        detected_count[int(det[0])]+=1

      print("detected: ",detected_count )
      print("counted: ", vehicle_count)
  video.release()
  out.release()

        

  return

In [ ]:
# Set up variables for running user's model
PATH_TO_VIDEO='/content/drive/MyDrive/processedTraffic5.mp4'   
PATH_TO_MODEL='/content/drive/MyDrive/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/drive/MyDrive/labelmap.txt'   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)


st = time.process_time()
# Run inferencing function!
tflite_detect_video(PATH_TO_MODEL, PATH_TO_VIDEO, PATH_TO_LABELS, min_conf_threshold)
et = time.process_time()
res = et - st
print("execution time is : ",res )

detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [0, 1, 3]
counted:  [1, 1, 3]
detected:  [0, 1, 3]
counted:  [1, 1, 3]
detected:  [0, 1, 3]
counted:  [1, 1, 3]
detected:  [0, 1, 3]
counted:  [1, 1, 3]
detected:  [0, 1

In [ ]:
!pip install filterpy==1.4.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=b0d6dd7342851ffda1c9fd27929e826f6f0566fa52df382bd5e025435ae2e599
  Stored in directory: /root/.cache/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
!ls
import os
from google.colab import files
import cv2
import numpy as np
import glob
import random
import importlib.util
from sortalgo import Sort
from tensorflow.lite.python.interpreter import Interpreter


import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
### Define function for inferencing with TFLite model and displaying results

def tflite_detect_video_sort(modelpath, vidpath, lblpath, min_conf=0.5, savepath='/content/drive/MyDrive/results'):


  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  video=cv2.VideoCapture(vidpath)
  imW = video.get(cv2.CAP_PROP_FRAME_WIDTH)
  imH = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  fps = video.get(cv2.CAP_PROP_FPS) 
  out = cv2.VideoWriter('output_video5s.mp4', fourcc, fps, (int(imW), int(imH)), isColor=True)

  vehicle_count = [0 for _ in range (3) ]
  
  max_num=0
  sort = Sort(max_age=1, min_hits=3, iou_threshold=0.3)

  # Loop over every image and perform detection
  while(video.isOpened()):
      detected_count = [0 for _ in range (3) ]
      ret,frame = video.read()
      if not ret:
        print("End of video reached!")
        break     
      frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame_resized = cv2.resize(frame_rgb, (width, height))
      input_data = np.expand_dims(frame_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections=[]
      
      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (10, 255, 0), 4)
              #crop the object from the image
              
              #cropped = cv2.cvtColor(cropped,cv2.COLOR_BGR2RGB)
              #plt.figure(figsize=(12,16))
              #plt.imshow(cropped)
              #plt.show()
              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(frame, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(frame, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text
              detections.append([xmin,ymin,xmax,ymax,scores[i],classes[i]])
      detected= np.vstack(detections)
      res= sort.update(detected)
      boxes_track = res[:,:-2]
      boxes_ids = res[:,-2].astype(int)
      boxes_classes= res[:,-1].astype(int)
      matrix = np.column_stack((boxes_ids, boxes_classes))
     # print(boxes_track, boxes_ids,boxes_classes)
      matrix_mask= matrix[:,0] > max_num
      if (np.any(matrix_mask == True)):
        outputs=matrix[matrix_mask]
        max_num=np.max(outputs[:,0])
        for classx in outputs[:,1]:
          vehicle_count[classx] +=1
      frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
      out.write(frame)
      
      for i , det in enumerate(detections):
        detected_count[int(det[5])]+=1

      print("detected: ",detected_count )
      print("counted: ", vehicle_count)
  video.release()
  out.release()

        

  return

 1047635422-preview.mp4      models		     processedTraffic5.mp4
 1097927181-preview.mp4      output_video1.mp4	     __pycache__
 1097929093-preview.mp4      output_video1s.mp4      sortalgo.py
 calculate_map_cartucho.py   output_video2.mp4	     test
'Colab Notebooks'	     output_video2s.mp4      training
 custom_model_lite	     output_video3.mp4	     yolotest-3
 labelmap.txt		     processedTraffic1.mp4
 mAP			     processedTraffic2.mp4


In [ ]:
# Set up variables for running user's model
PATH_TO_VIDEO='/content/drive/MyDrive/processedTraffic5.mp4'   
PATH_TO_MODEL='/content/drive/MyDrive/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/drive/MyDrive/labelmap.txt'   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
import time

st = time.process_time()
# Run inferencing function!
tflite_detect_video_sort(PATH_TO_MODEL, PATH_TO_VIDEO, PATH_TO_LABELS, min_conf_threshold)
et = time.process_time()
res = et - st
print("execution time is : ",res )

detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 1, 3]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [1, 1, 3]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [1, 2, 2]
counted:  [2, 3, 4]
detected:  [0, 1, 3]
counted:  [2, 3, 4]
detected:  [0, 1, 3]
counted:  [2, 3, 4]
detected:  [0, 1, 3]
counted:  [2, 3, 4]
detected:  [0, 1, 3]
counted:  [2, 4, 7]
detected:  [0, 1

#MAP calculator
 

In [ ]:
%%bash
cat <<EOF >> /content/drive/MyDrive/labelmap.txt
bus
car
lorry
EOF

In [ ]:
%%bash

cd /content/drive/MyDrive/mAP
rm input/detection-results/* 
rm input/ground-truth/* 
rm input/images-optional/* 
#wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

rm: cannot remove 'input/ground-truth/backup': Is a directory


In [ ]:
!cp /content/drive/MyDrive/test/* /content/drive/MyDrive/mAP/input/images-optional # Copy images and xml files
!mv /content/drive/MyDrive/mAP/input/images-optional/*.xml /content/drive/MyDrive/mAP/input/ground-truth/  # Move xml files to the appropriate folder

In [ ]:
!python /content/drive/MyDrive/mAP/scripts/extra/convert_gt_xml.py

Conversion completed!


In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/drive/MyDrive/test'   # Path to test images folder
PATH_TO_MODEL='/content/drive/MyDrive/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/drive/MyDrive/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/drive/MyDrive/mAP/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(1000, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

Starting inference on 989 images...
Finished inferencing!


In [ ]:
!cd /content/drive/MyDrive/mAP
!ls
!python calculate_map_cartucho.py --labels=/content/drive/MyDrive/labelmap.txt

calculate_map_cartucho.py  LICENSE	     main.py	scripts
input			   main_modified.py  README.md
Calculating mAP at 0.50 IoU threshold...
55.75% = bus AP 
56.74% = car AP 
77.03% = lorry AP 
mAP = 63.17%
Calculating mAP at 0.55 IoU threshold...
50.93% = bus AP 
47.06% = car AP 
72.56% = lorry AP 
mAP = 56.85%
Calculating mAP at 0.60 IoU threshold...
48.55% = bus AP 
36.75% = car AP 
67.97% = lorry AP 
mAP = 51.09%
Calculating mAP at 0.65 IoU threshold...
45.78% = bus AP 
27.24% = car AP 
60.67% = lorry AP 
mAP = 44.56%
Calculating mAP at 0.70 IoU threshold...
38.24% = bus AP 
15.21% = car AP 
51.92% = lorry AP 
mAP = 35.13%
Calculating mAP at 0.75 IoU threshold...
28.76% = bus AP 
8.17% = car AP 
41.64% = lorry AP 
mAP = 26.19%
Calculating mAP at 0.80 IoU threshold...
20.41% = bus AP 
2.64% = car AP 
28.62% = lorry AP 
mAP = 17.22%
Calculating mAP at 0.85 IoU threshold...
12.37% = bus AP 
0.48% = car AP 
16.45% = lorry AP 
mAP = 9.77%
Calculating mAP at 0.90 IoU threshold...
7.25% = bus 